In [ ]:
# cd C:\Users\Venuxk\Projects\data-visualizer
# conda activate visualizer
# jupyter notebook test.ipynb

In [2]:
import geoviews as gv
import holoviews as hv
import param

gv.help(gv.Path)
# hv.help(hv.Table)
help(param.List)
# hv.help(hv.streams.Pipe)

Path

Online example: http://holoviews.org/reference/elements/bokeh/Path.html

-------------
Style Options
-------------

	alpha, cmap, color, hover_alpha, hover_color, hover_line_alpha, hover_line_cap, hover_line_color, hover_line_dash, hover_line_join, hover_line_width, line_alpha, line_cap, line_color, line_dash, line_join, line_width, muted, muted_alpha, muted_color, muted_line_alpha, muted_line_cap, muted_line_color, muted_line_dash, muted_line_join, muted_line_width, nonselection_alpha, nonselection_color, nonselection_line_alpha, nonselection_line_cap, nonselection_line_color, nonselection_line_dash, nonselection_line_join, nonselection_line_width, selection_alpha, selection_color, selection_line_alpha, selection_line_cap, selection_line_color, selection_line_dash, selection_line_join, selection_line_width, visible

(Consult bokeh's documentation for more information.)

------------
Plot Options
------------

The plot options are the parameters of the plotting class:

Parameters

In [1]:
import rioxarray
import geoviews as gv
import cartopy
import rasterio
import xarray

data = rioxarray.open_rasterio(".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.asc").rename({"x": "Longitude", "y": "Latitude"})

# # data.rio.write_crs("epsg:32148", inplace = True)
# elwha_crs = rasterio.crs.CRS.from_dict({
#     "proj": "lcc",
#     "lat_1": 47.5,
#     "lat_2": 48.73333333333333,
#     "lon_0": -120.8333333333333,
#     "lat_0": 47.0,
#     "x_0": 500000.0,
#     "y_0": 0.0,
#     "units": "m",
#     "datum": "NAD83",
#     "ellps": "GRS80",
#     "no_defs": True,
#     "type": "crs"
# })
# data.rio.write_crs(elwha_crs, inplace = True)
# data.rio.to_raster(
#     raster_path = ".\data\Elwha\ew11_may_dem_1m.tif",
#     driver = "GTiff"
# )

data

# dataset = rioxarray.open_rasterio(".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.tif")
# data = gv.Dataset(dataset, ["x", "y"], "value")
# plot = data.to(gv.Image)

# plot = gv.load_tiff(
#     ".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.tif",
#     crs = cartopy.crs.epsg(26914),
#     nan_nodata = True
# )
# gv.tile_sources.Wikipedia * plot

# data = xarray.open_dataset(".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.tif").rename({"x": "Longitude", "y": "Latitude"})
# dataset = gv.Dataset(
#     data = data,
#     # kdims = ["x", "y"],
#     kdims = ["Longitude", "Latitude"],
# #     vdims = "Elevation (meters)"
# )
# data
# plot = dataset.to(
#     gv.Image,
# #     kdims = ["Longitude", "Latitude"]
# #     vdims = "Elevation (meters)"
#     vdims = "z"
# ).options(
#     # cmap = "Turbo",
#     tools = ["hover"],
#     alpha = 0.5
# )
# # gv.tile_sources.Wikipedia * plot
# type(plot)
# type(dataset.clone(datatype=['xarray']).data)

<xarray.DataArray (band: 1, Latitude: 1261, Longitude: 1661)>
[2094521 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * Longitude    (Longitude) float64 2.96e+05 2.96e+05 ... 2.977e+05 2.977e+05
  * Latitude     (Latitude) float64 1.319e+05 1.318e+05 ... 1.306e+05 1.306e+05
    spatial_ref  int32 0
Attributes:
    _FillValue:    -9999.0
    scale_factor:  1.0
    add_offset:    0.0

In [3]:
import rasterio

# dataset = rasterio.open(".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.asc")
# band1 = dataset.read(1)
# dataset

dataset = rasterio.open(".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.tif")
dataset.crs

In [8]:
import geoviews as gv

t = gv.load_tiff(".\data\Elwha\Digital Elevation Models (DEMs)\ew11_may_dem_1m.tif", nan_nodata=True)
gv.tile_sources.Wikipedia * t

:Overlay
   .WMTS.I  :WMTS   [Longitude,Latitude]
   .Image.I :Image   [x,y]   (z)

In [9]:
import pandas as pd
import geopandas
import rasterio

# dataframe = pd.read_csv(
#     "C:\Users\Venuxk\Projects\data-visualizer\data\Elwha\Transects\ew_lines.txt",
#     header = None,
#     names = ["Point_ID", "Easting_m", "Northing_m", "Placeholder_Col_Name"],
#     usecols = [0, 1, 2]
# )
features_list = []
with open(".\data\Elwha\Transects\ew_lines.txt", "r") as file:
    transect_feature = None
    for i, line in enumerate(file):
        [point_id, x, y, _] = line.split(",")
        if transect_feature is None:
            # Initialize a new transect feature.
            id = int("".join([char for char in point_id if char.isdigit()]))
            transect_feature = {
                "type": "Feature",
                "properties": {"transect_id": id},
                "geometry": {
                    "type": "LineString",
                    "coordinates": []
                }
            }
            # Add the transect's start point.
            transect_feature["geometry"]["coordinates"].append([float(x), float(y)])
        else:
            # Add the transect's end point.
            transect_feature["geometry"]["coordinates"].append([float(x), float(y)])
            # Save the transect to the FeatureCollection.
            features_list.append(transect_feature)
            # Reset the feature for the next transect.
            transect_feature = None
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.from_features.html
gdf = geopandas.GeoDataFrame.from_features(
    {"type": "FeatureCollection", "features": features_list},
    crs = rasterio.crs.CRS.from_dict({
        "proj": "lcc",
        "lat_1": 47.5,
        "lat_2": 48.73333333333333,
        "lon_0": -120.8333333333333,
        "lat_0": 47.0,
        "x_0": 500000.0,
        "y_0": 0.0,
        "units": "m",
        "datum": "NAD83",
        "ellps": "GRS80",
        "no_defs": True,
        "type": "crs"
    })
)
# https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_file.html#geopandas.GeoDataFrame.to_file
gdf.to_file(".\data\Elwha\Transects\ew_lines.geojson", driver = "GeoJSON")

In [4]:
import geoviews as gv
import holoviews as hv
from holoviews import streams, opts
from holoviews.plotting.links import DataLink

points = gv.Points(data = [[0,0], [1,1]]).opts(active_tools = ["point_draw"])
len(points.array())
# curve_stream = streams.CurveEdit(data=points.columns(), source=points, style={'color': 'black', 'size': 10})
# table = hv.Table(points).opts(editable=True)
# DataLink(points, table)

# (points + table).opts(
#     opts.Table(editable=True))

2